<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Homework/4_Exercise_Representation_2/PH_4_HW_Representation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install transformers

In [2]:
#imports
import pickle
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf
import torch
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [3]:
#Mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Load lemmatized data as .pkl
with open('/content/drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Newsgroup/data_lemma.pkl', 'rb') as f:
    data_lemma = pickle.load(f)

lemma_df = pd.DataFrame(data_lemma, columns=["text"])

In [5]:
lemma_df.head(5)

,text
0,car wonder enlighten car see day door sport ca...
1,clock poll final final clock report accelerati...
2,question folk mac plus finally give ghost week...
3,weitek nntp post host amber ssd csd harris com...
4,shuttle launch question article cowcb world st...


In [6]:
# Generate corpus
corpus_gen = [doc.split() for doc in lemma_df["text"]]

# Word2Vec

In [7]:
# Initialize and train a ~gensim.models.word2vec.Word2Vec model
model = Word2Vec(corpus_gen, size=100, min_count=566)
#size = dimensionality of vector
#min_count: ignores all words with total frequency lower than this

In [8]:
#save model
model.save('drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Models/word2vec.model')

Print the sorted features of the model using model.wv.vocab.keys(). Compare them
with those from Exercise 3

In [9]:
model.wv.vocab.keys()

dict_keys(['car', 'wonder', 'see', 'day', 'look', 'early', 'call', 'small', 'body', 'know', 'model', 'year', 'history', 'info', 'mail', 'thank', 'bring', 'report', 'nntp', 'post', 'host', 'washington', 'edu', 'number', 'experience', 'send', 'message', 'speed', 'rate', 'add', 'card', 'disk', 'request', 'network', 'base', 'answer', 'guy', 'question', 'mac', 'give', 'start', 'life', 'way', 'new', 'machine', 'bit', 'maybe', 'anybody', 'expect', 'hear', 'suppose', 'access', 'price', 'line', 'like', 'go', 'display', 'probably', 'get', 'feel', 'well', 'great', 'good', 'opinion', 'people', 'use', 'take', 'size', 'money', 'hit', 'real', 'play', 'figure', 'actually', 'advance', 'email', 'news', 'time', 'truth', 'com', 'uucp', 'write', 'article', 'chip', 'far', 'low', 'level', 'stuff', 'pretty', 'nice', 'require', 'address', 'phone', 'information', 'thing', 'person', 'sense', 'world', 'clear', 'memory', 'error', 'yes', 'quote', 'software', 'check', 'right', 'value', 'set', 'code', 'tell', 'second

Show model.wv['car']. What is it doing? Why does it have this size?

In [10]:
print(model.wv['car'])
model.wv['car'].shape
# Shape because of the defined size paramter of model

[ 0.39827678 -0.6818796  -0.65525955 -0.6919702  -0.9987134  -1.7431556
  0.52977926  0.15701766 -0.03971322  0.01805263 -0.5385682   0.09339743
  0.01464276 -0.05242912 -1.393666   -0.769598   -0.08381373 -0.85941654
  1.6194606   0.09081569 -2.0508666  -0.1046214  -0.23145385  0.82549906
 -1.0034355   0.9810111  -0.36019957  0.09191548 -0.36628205 -0.23136732
  1.9034404   0.7712996  -0.94511837 -0.7561034  -0.5386708   0.6655813
 -0.78539747 -0.89321035  1.9307877   0.22916017  0.5282283   1.7821529
 -2.1904023  -0.50419116 -0.2395403   0.09688714  1.2904803  -0.27308095
  0.51264197 -0.2685298   0.88991725 -0.03252438 -0.36658874 -0.07369605
  0.583677   -1.3209583   0.73963356 -0.4844816   0.19089733  0.3969674
  0.21978311  0.6227351   0.45101014 -0.49244934  0.58836806  1.2121172
  2.1619654   0.08259585 -0.2859731  -1.5056658  -0.08731521 -1.0655843
  1.2676672   0.6774307  -2.0688941   0.04275506 -0.82391244  1.0568684
  1.4118186   1.2272408  -1.3312242   0.66444254  0.103444

(100,)

In [11]:
# Does the result make sense?
model.wv.most_similar('car')

[('bike', 0.643164873123169),
 ('buy', 0.5671544075012207),
 ('light', 0.5066849589347839),
 ('speed', 0.49451181292533875),
 ('friend', 0.4850934147834778),
 ('get', 0.4771081805229187),
 ('look', 0.4534294605255127),
 ('nice', 0.4513017535209656),
 ('hit', 0.44672441482543945),
 ('figure', 0.4439582824707031)]

- Some words seem more similar to car, then others (e.g. "buy", but not "figure")
- But overall the result makes sense I guess

Run model.wv.most_similar(positive=['bike', ' machine'], topn=1). What is it doing? Test
other combinations.

In [12]:
model.wv.most_similar(positive=['bike', 'machine'], topn=1)

[('buy', 0.6418911814689636)]

This method computes cosine similarity between a simple mean of the projection
weight vectors of the given words and the vectors for each word in the model.

In [13]:
print(model.wv.most_similar(positive=['bike', 'car'], topn=1))
print(model.wv.most_similar(positive=['jesus', 'car'], topn=1))
print(model.wv.most_similar(positive=['religion', 'man'], topn=1))
print(model.wv.most_similar(positive=['news', 'man'], topn=1))
print(model.wv.most_similar(positive=['news', 'woman'], topn=1))

[('buy', 0.6494540572166443)]
[('christ', 0.6020453572273254)]
[('christ', 0.7483876943588257)]
[('woman', 0.581000804901123)]
[('child', 0.5270392298698425)]


Generate a corpus as a list, where for each document the list entry consists of the list
of word2vec embeddings of the words in this document. Remove from it the empty
documents. Why were those generated? Aggregate the values at a document level
(i.e., one embedding per document) by averaging the word embeddings. Store the
result for all documents in a data frame and then in GoogleDrive as
WordtoVecModel.pkl.

In [50]:
corpus_2 = []

for doc in corpus_gen:
  words=[]
  for word in doc:
    try:
      words.append(model.wv[word])
    except KeyError: #skip words which are not in dict
      next
  if len(words)==0: #remove empty documents (doc where all words were not in dict)
    next
  else:
    corpus_2.append(np.mean(words, axis=0)) # average word embeddings per doc

In [57]:
# Create df
WordtoVecModel = pd.DataFrame(np.concatenate([corpus_2],axis=0))
WordtoVecModel.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.150655,-0.167528,0.000446,-0.369484,-0.497597,-0.512169,0.145186,0.049681,0.113843,0.013968,...,0.018139,0.670854,-0.124385,0.300157,-0.336985,0.171559,-0.229898,-0.054308,0.010837,-0.029526
1,0.371482,-0.053825,-0.038065,-0.213904,0.245400,0.324823,0.502939,-0.223659,0.002877,-0.100856,...,0.124590,-0.065208,-0.011630,-0.109283,-0.086218,0.122048,0.062208,-0.234205,0.395853,-0.021694
2,0.158555,0.251146,-0.029250,-0.194091,-0.125183,-0.195976,-0.293059,-0.109566,0.388214,-0.117620,...,-0.160956,0.232339,0.140396,-0.011282,-0.530027,0.019885,0.105696,-0.097548,0.082946,0.029190
3,0.189193,0.039397,-0.128204,0.084464,0.226446,0.123522,0.369446,-0.191700,0.192918,-0.320274,...,0.058099,0.140667,0.196520,-0.431090,-0.078227,0.079195,-0.031887,-0.181108,0.451697,-0.060556
4,-0.092418,0.090598,-0.152884,-0.393708,0.318162,0.271351,0.073015,-0.150181,0.249248,-0.300994,...,-0.194737,0.042304,0.344310,-0.132207,-0.398165,0.018271,0.287932,-0.142553,0.266741,0.213238


In [58]:
#Store the result for all documents in a data frame and then in GoogleDrive as WordtoVecModel.pkl.
WordtoVecModel.to_pickle('drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Models/WordtoVecModel.pkl')

In [62]:
# Test if storing worked
with open('drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Models/WordtoVecModel.pkl', 'rb') as f:
    WordtoVecModel = pickle.load(f)
WordtoVecModel.head()    

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.150655,-0.167528,0.000446,-0.369484,-0.497597,-0.512169,0.145186,0.049681,0.113843,0.013968,...,0.018139,0.670854,-0.124385,0.300157,-0.336985,0.171559,-0.229898,-0.054308,0.010837,-0.029526
1,0.371482,-0.053825,-0.038065,-0.213904,0.245400,0.324823,0.502939,-0.223659,0.002877,-0.100856,...,0.124590,-0.065208,-0.011630,-0.109283,-0.086218,0.122048,0.062208,-0.234205,0.395853,-0.021694
2,0.158555,0.251146,-0.029250,-0.194091,-0.125183,-0.195976,-0.293059,-0.109566,0.388214,-0.117620,...,-0.160956,0.232339,0.140396,-0.011282,-0.530027,0.019885,0.105696,-0.097548,0.082946,0.029190
3,0.189193,0.039397,-0.128204,0.084464,0.226446,0.123522,0.369446,-0.191700,0.192918,-0.320274,...,0.058099,0.140667,0.196520,-0.431090,-0.078227,0.079195,-0.031887,-0.181108,0.451697,-0.060556
4,-0.092418,0.090598,-0.152884,-0.393708,0.318162,0.271351,0.073015,-0.150181,0.249248,-0.300994,...,-0.194737,0.042304,0.344310,-0.132207,-0.398165,0.018271,0.287932,-0.142553,0.266741,0.213238


Find the most similar words to the embedding representing the first document from 10.
Use cosine() to determine the document on the corpus most similar to it.

In [89]:
# find most similar word to first document in corpus (averaged embedding) in model.wv.vectors 
index = np.argmax(model.wv.cosine_similarities(WordtoVecModel.iloc[0,:], model.wv.vectors))
print(index)
model.wv.index2word[index]

63


'car'

In [93]:
# find most similar doc to first document in corpus (averaged embedding) 
index = np.argmax(model.wv.cosine_similarities(WordtoVecModel.iloc[0,:], WordtoVecModel.iloc[1:,:]))+1 #index +1 because we excluded first row of df (similarity=1)
print(index)
WordtoVecModel.iloc[[index]] 

4976


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
4976,0.256408,0.064898,0.008776,-0.122782,-0.417759,-0.328052,-0.008624,-0.155028,0.404104,0.059023,...,-0.02657,0.731384,-0.114866,0.183558,-0.328125,0.255573,-0.220748,-0.242193,-0.105913,-0.055683


In [94]:
# Check if most similar word to most similar doc in corpus (averaged embedding) is also "car" in model.wv.vectors 
index = np.argmax(model.wv.cosine_similarities(WordtoVecModel.iloc[index,:], model.wv.vectors))
print(index)
model.wv.index2word[index]

63


'car'

# Doc2vec

Run gensim doc2vec model on corpus_gen (vector_size=100, min_count=566). (Note:
Doc2vec requires tagged documents). Determine the embedding representation for the
first document as well as the document in the dataset most similar to it. Compare it to
the result in 10. Generate the final corpus as a data frame and store it in GoogleDrive
as DoctoVecModel.pkl..

In [104]:
# Tag Documents
corpus_gen_t = []
for i, tokens in enumerate(corpus_gen):
  corpus_gen_t.append(TaggedDocument(tokens, [i]))

In [107]:
# initialize model 
d_model = Doc2Vec(corpus_gen_t, vector_size=100, min_count=566)

In [126]:
# infer vector (embedding representation) of first document 
inf_vec = d_model.infer_vector(corpus_gen_t[0][0])

In [128]:
# Top 10 most similar 
sims = d_model.docvecs.most_similar([inf_vec], topn=10)
sims

[(0, 0.8106570243835449),
 (3819, 0.792576014995575),
 (3550, 0.7913380861282349),
 (445, 0.7799179553985596),
 (4600, 0.772595226764679),
 (9196, 0.7702265381813049),
 (7861, 0.7685385346412659),
 (11288, 0.7679082751274109),
 (4456, 0.7658605575561523),
 (1047, 0.7613057494163513)]

In [134]:
vec_ms = d_model.infer_vector(corpus_gen_t[sims[1][0]].words)
vec_ms

array([ 0.03939803,  0.0030868 , -0.13779815,  0.1030381 , -0.23047528,
       -0.0877317 , -0.02308975,  0.07620917,  0.00743653,  0.16874756,
        0.11099327, -0.21795046,  0.00810836, -0.16032165, -0.23383787,
       -0.03893895,  0.03716713, -0.1441815 ,  0.17089555,  0.08035015,
       -0.13690284, -0.06241753, -0.27806956, -0.04496985, -0.22355708,
        0.24847798, -0.17059168,  0.16203728,  0.0994089 ,  0.2632886 ,
        0.20338736,  0.14723602, -0.1910176 , -0.02871284, -0.06476048,
        0.19563058, -0.23224536, -0.09760678,  0.4979108 ,  0.04846481,
        0.10950635, -0.04535366, -0.42864633,  0.00094055, -0.16082078,
        0.06794269,  0.11982191, -0.11386397, -0.18312824, -0.05350937,
        0.12591149,  0.22761072, -0.08456829, -0.02280084,  0.01922724,
        0.16712774, -0.1322307 , -0.05295815, -0.12094194,  0.19907603,
       -0.04262339,  0.00108411,  0.12258077, -0.13629547,  0.03827178,
       -0.07117741,  0.36608   ,  0.2506777 ,  0.033354  , -0.09

In [140]:
print(lemma_df.iloc[sims[1][0]].text)

ford probe opinion center jeremy mereness zonker cmu edu say offer opinions ford probe ala long run repair record reliability mileage fix buy car month aim little big typical small car little power consider probe accord corolla probe young bunch opinions magazine rating book place excellent value btw rear wheel drive design thank advance read normally read group look valentine radar information sigh maybe faq come post buy probe pep july mile debate sentra stealth prelude celica check month consumer report previous probe record criterion fun car ab airbag think turbo check insurance people rule tri star cars eclipse talon air bag ditto mustang abs fall pocket rocket category good car buy roll like like prelude dash instrumentation weird small non passenger space need go celica underpowere loaded option somewhat overprice non turbo form consider airbag look rwd virtue corolla entered mind look mitsubishi dodge stealth list exhausted buy probe car design different early year early reliab

Result from Doc2Vec seems to be better fitting then the averaged w2v

In [151]:
# Generate the final corpus as a data frame and store it in GoogleDrive as DoctoVecModel.pkl.
corpus_3 = [d_model.infer_vector(doc) for doc, doc_id in corpus_gen_t]

In [156]:
# Create df
DoctoVecModel = pd.DataFrame(np.concatenate([corpus_3],axis=0))
DoctoVecModel.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.019556,-0.002473,0.027714,0.002049,-0.051203,-0.023470,0.038007,0.019787,0.011376,0.057402,...,-0.029897,0.076302,-0.080510,0.003817,-0.102602,-0.012824,0.039288,0.013901,0.035912,0.075591
1,0.089255,0.022071,-0.012584,-0.059912,0.039230,0.036106,0.045976,0.028675,-0.027386,0.116780,...,0.020228,-0.002983,0.013145,0.066479,-0.048470,0.031540,0.013921,-0.070716,-0.007303,0.043357
2,0.106090,0.177577,0.028048,-0.105931,-0.024951,-0.101361,-0.131309,-0.050143,0.149129,0.048127,...,-0.064408,-0.060442,0.075986,0.010480,-0.047085,-0.038759,0.115350,-0.124995,0.063829,0.043481
3,-0.028032,0.084766,-0.015731,-0.013414,-0.051138,-0.046498,0.016996,-0.004060,0.041146,0.027825,...,0.049122,-0.030293,-0.045480,-0.033850,-0.063363,-0.015023,0.050747,0.016347,-0.002227,-0.012333
4,-0.038871,-0.012824,0.015652,-0.141965,-0.028991,0.004396,0.011437,0.004229,0.056814,-0.051368,...,-0.036690,0.005222,0.023500,0.040639,-0.097264,-0.001881,0.111407,-0.009567,0.009186,0.020623


In [160]:
#Store the result for all documents in a data frame and then in GoogleDrive as WordtoVecModel.pkl.
DoctoVecModel.to_pickle('drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Models/DoctoVecModel.pkl')

In [161]:
# Test if storing worked
with open('drive/MyDrive/Colab_Notebooks/02_HWR/00_data/Models/DoctoVecModel.pkl', 'rb') as f:
    DoctoVecModel = pickle.load(f)
DoctoVecModel.head()    

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.019556,-0.002473,0.027714,0.002049,-0.051203,-0.023470,0.038007,0.019787,0.011376,0.057402,...,-0.029897,0.076302,-0.080510,0.003817,-0.102602,-0.012824,0.039288,0.013901,0.035912,0.075591
1,0.089255,0.022071,-0.012584,-0.059912,0.039230,0.036106,0.045976,0.028675,-0.027386,0.116780,...,0.020228,-0.002983,0.013145,0.066479,-0.048470,0.031540,0.013921,-0.070716,-0.007303,0.043357
2,0.106090,0.177577,0.028048,-0.105931,-0.024951,-0.101361,-0.131309,-0.050143,0.149129,0.048127,...,-0.064408,-0.060442,0.075986,0.010480,-0.047085,-0.038759,0.115350,-0.124995,0.063829,0.043481
3,-0.028032,0.084766,-0.015731,-0.013414,-0.051138,-0.046498,0.016996,-0.004060,0.041146,0.027825,...,0.049122,-0.030293,-0.045480,-0.033850,-0.063363,-0.015023,0.050747,0.016347,-0.002227,-0.012333
4,-0.038871,-0.012824,0.015652,-0.141965,-0.028991,0.004396,0.011437,0.004229,0.056814,-0.051368,...,-0.036690,0.005222,0.023500,0.040639,-0.097264,-0.001881,0.111407,-0.009567,0.009186,0.020623
